In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker as mtick
plt.style.use('ggplot')
import seaborn as sns
pd.options.display.float_format = '{:,.0f}'.format
import dataframe_image as dfi
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
lm = LinearRegression()
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
encoder = OneHotEncoder(sparse_output=False)
from sklearn.exceptions import ConvergenceWarning
import warnings

In [2]:
Ames = pd.read_csv('Ames_HousePrice.csv')

In [3]:
#drop extra unnamed column
Ames = Ames.drop('Unnamed: 0', axis=1)

In [4]:
Ames_Data_Types = pd.DataFrame({'Column': Ames.columns, 'Data Type': Ames.dtypes.values})
#Ames_Data_Types.to_csv('Ames_Data_Types.csv', index=False)  #Export, only needed once

In [5]:
Ames = Ames.drop_duplicates(keep = 'first')
print('\nDuplicate Rows:\n', Ames[Ames.duplicated(keep=False)]) #duplicate rows


Duplicate Rows:
 Empty DataFrame
Columns: [PID, GrLivArea, SalePrice, MSSubClass, MSZoning, LotFrontage, LotArea, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, PoolQC, Fence, MiscFeature, MiscVal, MoSold, YrSold, SaleType, SaleCondition]
Index: []

[0 rows 

In [6]:
Ames_Missing_Values = (
    Ames.isnull().sum()[Ames.isnull().sum() != 0]
    .to_frame('MissingCount')
    .assign(MissingPercent=lambda x: 100 * x['MissingCount'] / len(Ames))
    .reset_index()
    .rename(columns={'index': 'ColumnName'})
)
Ames_Missing_Columns = Ames_Missing_Values['ColumnName']

In [7]:
Ames_Missing_Columns = Ames[Ames_Missing_Values['ColumnName']]
Ames_Missing_Columns_Numeric = Ames_Missing_Columns.select_dtypes(include=['number'])
Ames_Missing_Columns_Categorical = Ames_Missing_Columns.select_dtypes(include=['object', 'category'])

In [8]:
#Round 1 Imputation: Zoning & Quality Pivot Tables
Zoning_Shape_Pivot = Ames.pivot_table(
    index=['MSZoning', 'LotShape'],
    aggfunc={'LotFrontage': 'mean', 'LotArea': 'mean'}
).rename(columns={
    'LotFrontage': 'LotFrontage_Mean',
    'LotArea': 'LotArea_Mean'
})

OverallQual_Cond_Pivot = Ames.pivot_table(
    index=['OverallQual', 'OverallCond'],
    aggfunc={
        'Electrical': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
        'MasVnrType': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    }
).rename(columns={
    'Electrical': 'Electrical_Mode',
    'MasVnrType': 'MasVnrType_Mode'
})

#Merge Pivots into Ames
Ames = Ames.merge(Zoning_Shape_Pivot.reset_index(), on=['MSZoning', 'LotShape'], how='left')
Ames = Ames.merge(OverallQual_Cond_Pivot.reset_index(), on=['OverallQual', 'OverallCond'], how='left')

#Impute LotFrontage proportionally to LotArea
fallback_ratio = Ames['LotFrontage'].dropna().mean() / Ames['LotArea'].dropna().mean()
Ames['LotFrontage'] = Ames['LotFrontage'].fillna(
    (Ames['LotFrontage_Mean'] / Ames['LotArea_Mean'] * Ames['LotArea']).fillna(fallback_ratio * Ames['LotArea'])
)

#Impute Electrical
Ames['Electrical'] = Ames['Electrical'].fillna(Ames['Electrical_Mode'])
Ames['Electrical'] = Ames['Electrical'].fillna(Ames['Electrical'].dropna().mode().iloc[0])

#MasVnrType: “No Veneer” where area ≤ 0, else from pivot/mode
Ames.loc[Ames['MasVnrType'].isnull() & (Ames['MasVnrArea'] <= 0), 'MasVnrType'] = 'No Veneer'
Ames['MasVnrType'] = Ames['MasVnrType'].fillna(Ames['MasVnrType_Mode'])
Ames['MasVnrType'] = Ames['MasVnrType'].fillna(Ames['MasVnrType'].dropna().mode().iloc[0])

#Clean up Round 1 merge columns
Ames.drop(['LotFrontage_Mean', 'LotArea_Mean', 'Electrical_Mode', 'MasVnrType_Mode'], axis=1, inplace=True)

#Simple Fill for Remaining NAs
Ames.fillna({
    'Alley': 'No Alley Access',
    'BsmtCond': 'No Basement',
    'BsmtExposure': 'No Basement',
    'BsmtFinType1': 'No Basement',
    'BsmtQual': 'No Basement',
    'Fence': 'No Fence',
    'FireplaceQu': 'No Fireplace',
    'GarageType': 'No Garage',
    'MiscFeature': 'No Misc Feature',
    'PoolQC': 'No Pool'
}, inplace=True)

for col in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'MasVnrArea']:
    Ames[col] = Ames[col].fillna(0)
Ames['MasVnrArea'] = Ames['MasVnrArea'].replace({1: 0})  # correction

In [9]:
#Round 2 Imputation: Garage Pivot Tables
OverallQual_Cond_GarageType_Pivot = Ames.pivot_table(
    index=['OverallQual', 'OverallCond', 'GarageType'],
    aggfunc={
        'GarageFinish': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
        'GarageQual': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
        'GarageCond': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    }
).rename(columns={
    'GarageFinish': 'GarageFinish_Mode',
    'GarageQual': 'GarageQual_Mode',
    'GarageCond': 'GarageCond_Mode'
})

#Merge Pivot
Ames = Ames.merge(OverallQual_Cond_GarageType_Pivot.reset_index(),
                  on=['OverallQual', 'OverallCond', 'GarageType'], how='left')

#Imputations
Ames.loc[Ames['GarageType'] == 'No Garage', ['GarageFinish', 'GarageQual', 'GarageCond']] = 'No Garage'
Ames['GarageYrBlt'] = Ames['GarageYrBlt'].fillna(Ames['YearBuilt'])
Ames['GarageFinish'] = Ames['GarageFinish'].fillna(Ames['GarageFinish_Mode'])
Ames['GarageQual'] = Ames['GarageQual'].fillna(Ames['GarageQual_Mode'])
Ames['GarageCond'] = Ames['GarageCond'].fillna(Ames['GarageCond_Mode'])

#Fallbacks
for col in ['GarageFinish', 'GarageQual', 'GarageCond']:
    Ames[col] = Ames[col].fillna(Ames[col].dropna().mode().iloc[0])

#Drop merged columns
Ames.drop(['GarageFinish_Mode', 'GarageQual_Mode', 'GarageCond_Mode'], axis=1, inplace=True)

In [10]:
#BsmtFinType2 Imputation
Ames.loc[Ames['BsmtFinType1'] == 'No Basement', 'BsmtFinType2'] = 'No Basement'
Ames['BsmtFinType2'] = Ames['BsmtFinType2'].fillna(Ames['BsmtFinType2'].dropna().mode().iloc[0])

#Round 3 Imputation: Garage & Basement Pivot Tables
Garage_Qual_Cond_Type_Pivot = Ames.pivot_table(
    index=['GarageQual', 'GarageCond', 'GarageType'],
    aggfunc={
        'GarageArea': 'mean',
        'GarageCars': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    }
).rename(columns={
    'GarageArea': 'GarageArea_Mean',
    'GarageCars': 'GarageCars_Mode'
})

Basement_Qual_FinType1_FinType2_Pivot = Ames.pivot_table(
    index=['BsmtQual', 'BsmtFinType1', 'BsmtFinType2'],
    aggfunc={
        'BsmtFullBath': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
        'BsmtHalfBath': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    }
).rename(columns={
    'BsmtFullBath': 'BsmtFullBath_Mode',
    'BsmtHalfBath': 'BsmtHalfBath_Mode'
})

#Merge both pivots
Ames = Ames.merge(Garage_Qual_Cond_Type_Pivot.reset_index(),
                  on=['GarageQual', 'GarageCond', 'GarageType'], how='left')
Ames = Ames.merge(Basement_Qual_FinType1_FinType2_Pivot.reset_index(),
                  on=['BsmtQual', 'BsmtFinType1', 'BsmtFinType2'], how='left')

#Impute numeric + categorical
Ames['GarageArea'] = Ames['GarageArea'].fillna(Ames['GarageArea_Mean'])
Ames['GarageCars'] = Ames['GarageCars'].fillna(Ames['GarageCars_Mode'])
Ames['BsmtFullBath'] = Ames['BsmtFullBath'].fillna(Ames['BsmtFullBath_Mode'])
Ames['BsmtHalfBath'] = Ames['BsmtHalfBath'].fillna(Ames['BsmtHalfBath_Mode'])

#Global fallback fill for residual NaNs
for col in ['GarageArea', 'GarageCars', 'BsmtFullBath', 'BsmtHalfBath']:
    if Ames[col].isnull().any():
        if Ames[col].dtype.kind in 'biufc':
            Ames[col] = Ames[col].fillna(Ames[col].mean())
        else:
            Ames[col] = Ames[col].fillna(Ames[col].mode().iloc[0])

#Drop temporary pivot columns
Ames.drop([
    'GarageArea_Mean', 'GarageCars_Mode',
    'BsmtFullBath_Mode', 'BsmtHalfBath_Mode'
], axis=1, inplace=True)

In [11]:
#Missing Value Check
print('\nColumns with Missing Values:\n', Ames.isnull().sum()[Ames.isnull().sum() != 0]) #sum of missing values by column name


Columns with Missing Values:
 Series([], dtype: int64)


In [12]:
Ames_Columns_Numeric = Ames[[
    '1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'EnclosedPorch', 'Fireplaces', 'FullBath',
    'GarageArea', 'GarageCars', 'GarageYrBlt', 'GrLivArea', 'HalfBath', 'KitchenAbvGr',
    'LotArea', 'LotFrontage', 'LowQualFinSF', 'MasVnrArea', 'MiscVal', 'MoSold',
    'OpenPorchSF', 'PoolArea', 'ScreenPorch', 'TotalBsmtSF', 'TotRmsAbvGrd', 'WoodDeckSF',
    'YearBuilt', 'YearRemodAdd', 'YrSold'
]]
Ames_Columns_Ordinal = Ames[[ 
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual',
    'Electrical', 'ExterCond', 'ExterQual', 'Fence', 'FireplaceQu', 'Functional',
    'GarageCond', 'GarageFinish', 'GarageQual', 'HeatingQC', 'KitchenQual',
    'LandSlope', 'LotShape', 'OverallCond', 'OverallQual', 'PavedDrive',
    'PoolQC', 'Utilities'
]]
Ames_Columns_Nominal = Ames[[ 
    'Alley', 'BldgType', 'CentralAir', 'Condition1', 'Condition2',
    'Exterior1st', 'Exterior2nd', 'Foundation', 'GarageType', 'Heating',
    'HouseStyle', 'LandContour', 'LotConfig', 'MasVnrType', 'MiscFeature',
    'MSSubClass', 'MSZoning', 'Neighborhood', 'RoofMatl', 'RoofStyle',
    'SaleCondition', 'SaleType', 'Street'
]].astype(str)
assert Ames_Columns_Numeric.index.equals(Ames_Columns_Nominal.index)

In [13]:
value_count_list = []
for col in Ames_Columns_Ordinal.columns:
    counts = Ames_Columns_Ordinal[col].value_counts(dropna=False)
    temp_df = pd.DataFrame({
        'Column Name': col,
        'Value': counts.index,
        'Value Count': counts.values
    })
    value_count_list.append(temp_df)
Ames_Ordinal_ValueCounts = pd.concat(value_count_list, ignore_index=True)
#Ames_Ordinal_ValueCounts.to_csv('Ames_Ordinal_ValueCounts.csv', index=False)

In [14]:
#Import Ordinal Value Ordering Legend created based off of Ames_Ordinal_ValueCounts export
Ames_Ordinal_Legend = pd.read_csv('Ames_Ordinal_Legend.csv')

#Convert already numeric ordinal values to string in original Dataset
ordinal_cols = Ames_Ordinal_Legend['Name'].unique()
for col in ordinal_cols:
    if col in Ames.columns:
        Ames[col] = Ames[col].astype(str)
# Loop through columns that are in the ordinal legend
for col in Ames.columns:
    if col in Ames_Ordinal_Legend['Name'].unique():
        # Create the mapping dictionary for this column
        mapping_dict = Ames_Ordinal_Legend.loc[Ames_Ordinal_Legend['Name'] == col,
                                               ['Value', 'Value_Order']].set_index('Value')['Value_Order'].to_dict()
        # Map values in Ames[col] using this dictionary
        Ames[col] = Ames[col].map(mapping_dict)

#Missing Value Check
print('\nColumns with Missing Values:\n', Ames.isnull().sum()[Ames.isnull().sum() != 0]) #sum of missing values by column name
print('\nRows with Missing Values:\n', Ames[Ames.isnull().any(axis=1)]) #show rows with missing values

# Check the data types and unique values
ordinal_cols = Ames_Ordinal_Legend['Name'].unique() # List of ordinal columns
# Check dtype and unique values as plain Python ints
for col in ordinal_cols:
    if col in Ames.columns:
        unique_vals = [int(v) for v in sorted(Ames[col].dropna().unique())]
        print(f"{col}: dtype={Ames[col].dtype}, unique values={unique_vals}")


Columns with Missing Values:
 Series([], dtype: int64)

Rows with Missing Values:
 Empty DataFrame
Columns: [PID, GrLivArea, SalePrice, MSSubClass, MSZoning, LotFrontage, LotArea, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, PoolQC, Fence, MiscFeature, Mis

In [15]:
#Update with new Ames Data
Ames_Columns_Ordinal = Ames[[ 
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual',
    'Electrical', 'ExterCond', 'ExterQual', 'Fence', 'FireplaceQu', 'Functional',
    'GarageCond', 'GarageFinish', 'GarageQual', 'HeatingQC', 'KitchenQual',
    'LandSlope', 'LotShape', 'OverallCond', 'OverallQual', 'PavedDrive',
    'PoolQC', 'Utilities'
]]
#Combination Table of Numeric & Ordinal Columns
Ames_Columns_Numeric_Ordinal = Ames[
    list(Ames_Columns_Numeric.columns) + list(Ames_Columns_Ordinal.columns)
]
#Combination Table of Nominal & Ordinal Columns
Ames_Columns_Nominal_Ordinal = Ames[
    list(Ames_Columns_Nominal.columns) + list(Ames_Columns_Ordinal.columns)
]

In [16]:
Y = Ames['SalePrice']
lm = LinearRegression()
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [17]:
encoder_nominal = OneHotEncoder(drop='first', sparse_output=False)
encoder_ordinal = OneHotEncoder(drop='first', sparse_output=False)
Nominal_cols = Ames_Columns_Nominal.columns
Ordinal_cols = Ames_Columns_Ordinal.columns
Numeric_cols = Ames_Columns_Numeric.columns

# --- One-Hot Encode Nominal ---
Ames_Optimal_Nominal_One_Hot = encoder_nominal.fit_transform(Ames[Nominal_cols])
Ames_Optimal_Nominal_One_Hot_df = pd.DataFrame(
    Ames_Optimal_Nominal_One_Hot,
    columns=encoder_nominal.get_feature_names_out(Nominal_cols),
    index=Ames.index
)

#One-Hot Encode Ordinal
Ames_Optimal_Ordinal_One_Hot = encoder_ordinal.fit_transform(Ames[Ordinal_cols])
Ames_Optimal_Ordinal_One_Hot_df = pd.DataFrame(
    Ames_Optimal_Ordinal_One_Hot,
    columns=encoder_ordinal.get_feature_names_out(Ordinal_cols),
    index=Ames.index
)

#Merge Nominal & Ordinal Encodings
Ames_Optimal_Categorical_One_Hot = pd.merge(
    Ames_Optimal_Nominal_One_Hot_df,
    Ames_Optimal_Ordinal_One_Hot_df,
    left_index=True,
    right_index=True,
    how='left'
)

#Merge with Numeric Features
Ames_MLR_Optimal_Columns = pd.merge(
    Ames[Numeric_cols],
    Ames_Optimal_Categorical_One_Hot,
    left_index=True,
    right_index=True,
    how='left'
)

#Model & Evaluation
lm = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=2)
X = Ames_MLR_Optimal_Columns
lm.fit(X, Y)
r2 = cross_val_score(lm, X, Y, cv=kf, scoring='r2')

#Output
print(f'Nominal: {Nominal_cols}')
print(f'Ordinal: {Ordinal_cols}')
print(f'Numeric: {Numeric_cols}\n')
print(f"R² per fold (sorted): {', '.join(f'{score:.4f}' for score in sorted(r2))}")
print(f'R² Average: {r2.mean():.4f}')
print(f'Intercept: {lm.intercept_:,.2f}')

Nominal: Index(['Alley', 'BldgType', 'CentralAir', 'Condition1', 'Condition2',
       'Exterior1st', 'Exterior2nd', 'Foundation', 'GarageType', 'Heating',
       'HouseStyle', 'LandContour', 'LotConfig', 'MasVnrType', 'MiscFeature',
       'MSSubClass', 'MSZoning', 'Neighborhood', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'Street'],
      dtype='object')
Ordinal: Index(['BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual',
       'Electrical', 'ExterCond', 'ExterQual', 'Fence', 'FireplaceQu',
       'Functional', 'GarageCond', 'GarageFinish', 'GarageQual', 'HeatingQC',
       'KitchenQual', 'LandSlope', 'LotShape', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolQC', 'Utilities'],
      dtype='object')
Numeric: Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GarageYr

In [21]:
scaler_Y = StandardScaler()
scaler_X = StandardScaler()
Y_scaled = scaler_Y.fit_transform(Y.values.reshape(-1, 1)).flatten()
X_scaled = pd.DataFrame(
    scaler_X.fit_transform(Ames_MLR_Optimal_Columns),
    columns=Ames_MLR_Optimal_Columns.columns,
    index=Ames.index
)
#X_scaled = pd.concat([X_numeric_scaled, Ames_Optimal_Categorical_One_Hot], axis=1)

In [19]:
kf = KFold(n_splits=5, shuffle=True, random_state=2)
pipelines = {
    'Ridge': Pipeline(steps=[('regressor', Ridge())]),
    'Lasso': Pipeline(steps=[('regressor', Lasso(max_iter=100))]),
    'ElasticNet': Pipeline(steps=[('regressor', ElasticNet(max_iter=100))])
}
cv_results = {}
for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X_scaled, Y_scaled, cv=kf, scoring='r2', n_jobs=-1)
    cv_results[name] = {
        'mean_R2': float(round(scores.mean(), 4)),
        'fold_R2': [float(round(s, 4)) for s in scores]
    }
print('Pipeline Cross_Val_Score Results (alpha=1)')
cv_results

Pipeline Cross_Val_Score Results (alpha=1)


{'Ridge': {'mean_R2': 0.8955,
  'fold_R2': [0.8993, 0.926, 0.8076, 0.9217, 0.9227]},
 'Lasso': {'mean_R2': -0.0005,
  'fold_R2': [-0.0004, -0.0009, -0.001, -0.0001, -0.0001]},
 'ElasticNet': {'mean_R2': 0.2698,
  'fold_R2': [0.2794, 0.262, 0.285, 0.2611, 0.2614]}}

In [23]:
#Ridge Regression- Optimal Alpha via GridSearchCV
ridge = Ridge()
alpha_ridge_grid = np.linspace(100, 200, 100)

# Setup Grid Search for Ridge
ridge_grid = GridSearchCV(estimator=pipelines['Ridge'],
                          param_grid={'regressor__alpha': alpha_ridge_grid},
                          verbose=1)
ridge_grid.fit(X_scaled, Y_scaled)
 
#Best alpha & score
print('\nRidge Pipeline GridSearchCV Results\n')
ridge_best_alpha = ridge_grid.best_params_['regressor__alpha']
ridge_best_score = ridge_grid.best_score_
print(f"Best Alpha for Ridge: {ridge_best_alpha}")
print(f"Best CV Score: {round(ridge_best_score, 4)}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits

Ridge Pipeline GridSearchCV Results

Best Alpha for Ridge: 165.65656565656565
Best CV Score: 0.9154


In [22]:
#Ridge Regression- Optimal Alpha via RidgeCV
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)
ridge = Ridge()

ridge_cv = RidgeCV(alphas=np.linspace(1, 1000, 100), cv=kf, scoring='r2')
ridge_cv.fit(X_scaled, Y_scaled)

print('RidgeCV Results\n')
alpha_opt_ridge = ridge_cv.alpha_
print(f"Optimal Ridge Alpha: {alpha_opt_ridge:.4f}")

#Fit Ridge with optimal alpha
ridge.set_params(alpha=alpha_opt_ridge)
ridge.fit(X_scaled, Y_scaled)
# Coefficients as a Series with feature names
ridge_coefs = pd.Series(ridge.coef_, index=X_scaled.columns)
ridge.score(X_scaled, Y_scaled)

#Scoring & Coefficients
cv_score_ridge = cross_val_score(ridge, X_scaled, Y_scaled, cv=kf, scoring='r2')
print(f"Mean R²: {cv_score_ridge.mean():.4f};  ", f"Fold R² scores: {', '.join([f'{s:.4f}' for s in cv_score_ridge])}")

ridge_coefs_formatted = ridge_coefs.sort_values(key=abs, ascending=False)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
print("\nTop 20 largest coefficients (by absolute value):")
print(ridge_coefs_formatted.head(20))

#Zero Coefficients
zero_coefs_ridge = ridge_coefs[ridge_coefs == 0].index.tolist()
print(f"\nNumber of features with zero coefficient: {len(zero_coefs_ridge)}")
print("Zero coefficient features:")
print(zero_coefs_ridge)

RidgeCV Results

Optimal Ridge Alpha: 162.4545
Mean R²: 0.9060;   Fold R² scores: 0.8995, 0.9304, 0.8489, 0.9235, 0.9278

Top 20 largest coefficients (by absolute value):
GrLivArea               0.1579
TotalBsmtSF             0.0992
2ndFlrSF                0.0988
BsmtFinSF1              0.0950
1stFlrSF                0.0942
OverallQual_8           0.0933
YearBuilt               0.0785
OverallQual_9           0.0709
LotArea                 0.0634
OverallQual_7           0.0610
Neighborhood_NoRidge    0.0598
Fireplaces              0.0578
Neighborhood_StoneBr    0.0574
Condition2_PosN        -0.0555
MasVnrArea              0.0517
Neighborhood_Somerst    0.0486
BsmtExposure_4          0.0471
BsmtQual_5              0.0449
OverallQual_4          -0.0442
ExterQual_3             0.0422
dtype: float64

Number of features with zero coefficient: 0
Zero coefficient features:
[]


In [33]:
#Lasso Regression- Optimal Alpha via GridSearchCV
lasso = Lasso()
alpha_lasso_grid = np.logspace(-4, 2, 200)
 
# Setup Grid Search for Ridge
lasso_grid = GridSearchCV(estimator=pipelines['Lasso'],
                          param_grid={'regressor__alpha': alpha_lasso_grid},
                          verbose=1)
lasso_grid.fit(X_scaled, Y_scaled)
print('\nLasso Pipeline GridSearchCV Results\n')

#Best alpha & score
lasso_best_alpha = lasso_grid.best_params_['regressor__alpha']
lasso_best_score = lasso_grid.best_score_
print(f"Best Alpha for lasso: {lasso_best_alpha:.6f}")
print(f"Best CV Score {round(lasso_best_score, 4)}")

Fitting 5 folds for each of 200 candidates, totalling 1000 fits

Lasso Pipeline GridSearchCV Results

Best Alpha for lasso: 0.003697
Best CV Score 0.9161


In [32]:
#Lasso Regression- Optimal Alpha via GridSearchCV
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

lasso_cv = LassoCV(alphas=np.logspace(-4, 2, 200), cv=kf, n_jobs=-1, max_iter=5000)
lasso_cv.fit(X_scaled, Y_scaled)

print('LassoCV Results\n')
alpha_opt_lasso = lasso_cv.alpha_
print(f"Optimal Lasso Alpha: {alpha_opt_lasso:.4f}")

#Fit Lasso with optimal alpha
lasso = Lasso(alpha=alpha_opt_lasso, max_iter=5000)
lasso.fit(X_scaled, Y_scaled)

lasso_coefs = pd.Series(lasso.coef_, index=X_scaled.columns)
lasso_coefs_formatted = lasso_coefs.sort_values(key=abs, ascending=False)
pd.set_option('display.float_format', lambda x: f'{x:.10f}')

#Scores & Coefficients
cv_scores_lasso = cross_val_score(lasso, X_scaled, Y_scaled, cv=kf, scoring='r2')
print(f"\nMean R² Scores: {cv_scores_lasso.mean():.4f};  ", f"Fold R² scores: {', '.join([f'{s:.4f}' for s in cv_scores_lasso])}")

print("\nTop 20 largest coefficients:")
print(lasso_coefs_formatted.head(20))

#Zero Coefficients
zero_coefs_lasso = lasso_coefs[lasso_coefs == 0].index.tolist()
non_zero_coefs_lasso = lasso_coefs[~lasso_coefs.index.isin(zero_coefs_lasso)]
print(f"\nNumber of features with Non-Zero coefficient: {len(non_zero_coefs_lasso)}")
print(f"\nNumber of features with Zero coefficient: {len(zero_coefs_lasso)}")
print("Zero coefficient features:")
print(zero_coefs_lasso)

LassoCV Results

Optimal Lasso Alpha: 0.0032

Mean R² Scores: 0.9071;   Fold R² scores: 0.8997, 0.9333, 0.8433, 0.9286, 0.9305

Top 20 largest coefficients:
GrLivArea                0.3443732766
YearBuilt                0.1221810693
OverallQual_8            0.1193793021
BsmtFinSF1               0.1081139466
OverallQual_7            0.0922286512
TotalBsmtSF              0.0890149470
OverallQual_9            0.0868498834
Condition2_PosN         -0.0610585170
Neighborhood_NoRidge     0.0602745848
Neighborhood_StoneBr     0.0582235891
Neighborhood_Somerst     0.0572182012
LotArea                  0.0571305802
BsmtQual_5               0.0519388582
Fireplaces               0.0474824614
SaleCondition_Partial    0.0469644959
Neighborhood_NridgHt     0.0454528258
BsmtExposure_4           0.0437888291
MasVnrArea               0.0428205770
ExterQual_3              0.0425393713
Neighborhood_Crawfor     0.0422048458
dtype: float64

Number of features with Non-Zero coefficient: 176

Number of featur

In [39]:
#ElasticNet Regression- Optimal Alpha via GridSearchCV
l1_ratio = np.linspace(0.01, 1, 100)
alpha_elastic_grid = np.linspace(200, 350, 100)

#Setup Grid Search for Ridge
elasticnet_grid = GridSearchCV(estimator=pipelines['ElasticNet'],
                               param_grid={'regressor__alpha': alpha_elastic_grid,
                                           'regressor__l1_ratio': l1_ratio},
                               verbose=1)
elasticnet_grid.fit(X_scaled, Y_scaled)


print('\nElasticNet Pipeline GridSearchCV Results\n')

#Best alpha & score
elasticnet_best_rho = elasticnet_grid.best_params_['regressor__l1_ratio']
elasticnet_best_alpha = elasticnet_grid.best_params_['regressor__alpha']
elasticnet_best_score = elasticnet_grid.best_score_
print(f"Best Rho for ElasticNet: {elasticnet_best_rho:.4f}")
print(f"Best Alpha for ElasticNet: {elasticnet_best_alpha:.6f}")
print(f"Best CV Score {round(elasticnet_best_score, 4)}")

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits

ElasticNet Pipeline GridSearchCV Results

Best Rho for ElasticNet: 1.0000
Best Alpha for ElasticNet: 281.818182
Best CV Score 0.9161


In [41]:
#ElasticNet Regression- Optimal Alpha via ElasticNetCV
elastic_cv = ElasticNetCV(
    l1_ratio=np.linspace(0.01, 1, 100),
    alphas=np.linspace(.001, 2, 100),
    cv=5, max_iter=5000, n_jobs=-1
)
elastic_cv.fit(X_scaled, Y_scaled)

print('ElasticNetCV Results\n')
best_alpha_elastic = elastic_cv.alpha_
best_l1_elastic = elastic_cv.l1_ratio_
print(f"Optimal ElasticNet l1_ratio: {best_l1_elastic:.2f}")
print(f"Optimal ElasticNet Alpha: {best_alpha_elastic:.10f}")


#Build final ElasticNet model using best parameters
final_en = ElasticNet(alpha=best_alpha_elastic, l1_ratio=best_l1_elastic, max_iter=10000)
final_en.fit(X_scaled, Y_scaled)

#Scires
cv_scores_elasticnet = cross_val_score(final_en, X_scaled, Y_scaled, cv=kf, scoring='r2')
print(f"Mean CV R²: {cv_scores_elasticnet.mean():.4f};  ", f"Fold R² scores: {', '.join([f'{s:.4f}' for s in cv_scores_elasticnet])}\n")

#Coefficient Series
coef_elastic = pd.Series(final_en.coef_, index=X_scaled.columns)
zero_features_elastic = coef_elastic[coef_elastic == 0].index.tolist()
nonzero_features_elastic = coef_elastic[coef_elastic != 0]

top20_elastic = nonzero_features_elastic.reindex(nonzero_features_elastic.abs().sort_values(ascending=False).index).head(20)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
print("Top 20 largest non-zero coefficients (abs, 4 decimals):")
print(top20_elastic)

#Zero Coefficients
print(f"\nNumber of features with Non-Zero coefficient: {len(nonzero_features_elastic)}")
print(f"\nNumber of features with Zero coefficient: {len(zero_features_elastic)}")
print("Zero coefficient features:")
print(zero_features_elastic)

ElasticNetCV Results

Optimal ElasticNet l1_ratio: 0.02
Optimal ElasticNet Alpha: 0.0615757576
Mean CV R²: 0.9072;   Fold R² scores: 0.9007, 0.9320, 0.8492, 0.9250, 0.9292

Top 20 largest non-zero coefficients (abs, 4 decimals):
GrLivArea               0.1639
BsmtFinSF1              0.0969
OverallQual_8           0.0956
TotalBsmtSF             0.0948
2ndFlrSF                0.0946
1stFlrSF                0.0924
YearBuilt               0.0752
OverallQual_9           0.0730
OverallQual_7           0.0654
Neighborhood_NoRidge    0.0619
LotArea                 0.0578
Neighborhood_StoneBr    0.0577
Condition2_PosN        -0.0548
Fireplaces              0.0535
MasVnrArea              0.0503
BsmtQual_5              0.0501
Neighborhood_Somerst    0.0486
BsmtExposure_4          0.0466
Neighborhood_NridgHt    0.0434
ExterQual_3             0.0431
dtype: float64

Number of features with Non-Zero coefficient: 241

Number of features with Zero coefficient: 55
Zero coefficient features:
['BsmtHalfBa

In [45]:
#ElasticNet.5 Regression where Rho = .5- Optimal Alpha via GridSearchCV
l1_ratio_5 = [.5]
alpha_elastic5_grid = np.linspace(.0001, .1, 100)
 
#Setup Grid Search for Ridge
elasticnet5_grid = GridSearchCV(estimator=pipelines['ElasticNet'],
                               param_grid={'regressor__alpha': alpha_elastic5_grid,
                                           'regressor__l1_ratio': l1_ratio_5},
                               verbose=1)
elasticnet5_grid.fit(X_scaled, Y_scaled)

print('\nElasticNet Pipeline GridSearchCV Results (Rho = .5)\n')

#Best alpha & score
elasticnet5_best_rho = elasticnet5_grid.best_params_['regressor__l1_ratio']
elasticnet5_best_alpha = elasticnet5_grid.best_params_['regressor__alpha']
elasticnet5_best_score = elasticnet5_grid.best_score_
print(f"Best Rho for ElasticNet: {elasticnet5_best_rho:.4f}")
print(f"Best Alpha for ElasticNet: {elasticnet5_best_alpha:.6f}")
print(f"Best CV Score {round(elasticnet5_best_score, 4)}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits

ElasticNet Pipeline GridSearchCV Results (Rho = .5)

Best Rho for ElasticNet: 0.5000
Best Alpha for ElasticNet: 0.007164
Best CV Score 0.9162


In [44]:
#ElasticNetCV where Rho = .5
elastic5_cv = ElasticNetCV(
    l1_ratio=.5,
    alphas=np.linspace(.0001, .1, 100),
    cv=5,
    max_iter=5000,
    n_jobs=-1
)
elastic5_cv.fit(X_scaled, Y_scaled)

print('ElasticNetCV Results (Rho = .5)\n')
best_alpha_elastic5 = elastic5_cv.alpha_
best_l1_elastic5 = elastic5_cv.l1_ratio_
print(f"Optimal ElasticNet.5 l1_ratio: {best_l1_elastic5:.2f}")
print(f"Optimal ElasticNet.5 Alpha: {best_alpha_elastic5:.4f}")

#Build final ElasticNet model using best parameters
final_en = ElasticNet(alpha=best_alpha_elastic5, l1_ratio=best_l1_elastic5, max_iter=10000)
final_en.fit(X_scaled, Y_scaled)  #Fit final model

#Scores
cv_scores_elasticnet5 = cross_val_score(final_en, X_scaled, Y_scaled, cv=kf, scoring='r2')
print(f"Mean CV R²: {cv_scores_elasticnet5.mean():.4f};  ", f"Fold R² scores: {', '.join([f'{s:.4f}' for s in cv_scores_elasticnet5])}\n")

#Coefficients as a Series
coef_elastic5 = pd.Series(final_en.coef_, index=X_scaled.columns)

#Summary of coefficients
zero_features_elastic5 = coef_elastic5[coef_elastic5 == 0].index.tolist()
nonzero_features_elastic5 = coef_elastic5[coef_elastic5 != 0]

top20_elastic5 = nonzero_features_elastic5.reindex(nonzero_features_elastic5.abs().sort_values(ascending=False).index).head(20)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
print("Top 20 largest non-zero coefficients (abs, 4 decimals):")
print(top20_elastic5)

#Zero coefficients
print(f"\nNumber of features with Non-Zero coefficient: {len(nonzero_features_elastic5)}")
print(f"\nNumber of features with Zero coefficient: {len(zero_features_elastic5)}")
print("Zero coefficient features:")
print(zero_features_elastic5)

ElasticNetCV Results (Rho = .5)

Optimal ElasticNet.5 l1_ratio: 0.50
Optimal ElasticNet.5 Alpha: 0.0072
Mean CV R²: 0.9071;   Fold R² scores: 0.9000, 0.9331, 0.8436, 0.9283, 0.9308

Top 20 largest non-zero coefficients (abs, 4 decimals):
GrLivArea                0.3352
OverallQual_8            0.1182
YearBuilt                0.1160
BsmtFinSF1               0.1053
OverallQual_7            0.0918
TotalBsmtSF              0.0917
OverallQual_9            0.0866
Neighborhood_NoRidge     0.0603
Condition2_PosN         -0.0598
Neighborhood_StoneBr     0.0575
Neighborhood_Somerst     0.0557
LotArea                  0.0555
BsmtQual_5               0.0528
Fireplaces               0.0482
SaleCondition_Partial    0.0459
Neighborhood_NridgHt     0.0446
BsmtExposure_4           0.0442
MasVnrArea               0.0430
ExterQual_3              0.0428
Neighborhood_Crawfor     0.0422
dtype: float64

Number of features with Non-Zero coefficient: 165

Number of features with Zero coefficient: 131
Zero coef

In [46]:
#Set optimal hyperparameters
pipelines['Ridge'].set_params(regressor__alpha=ridge_best_alpha)
pipelines['Lasso'].set_params(regressor__alpha=lasso_best_alpha)
pipelines['ElasticNet'].set_params(regressor__alpha=elasticnet5_best_alpha, regressor__l1_ratio=0.5)

#Cross-validation
cv_results = {}
for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X_scaled, Y_scaled, cv=kf, scoring='r2', n_jobs=-1)
    cv_results[name] = {
        'mean_R2': float(round(scores.mean(), 4)),
        'fold_R2': [float(round(s, 4)) for s in scores]
    }
print('Pipeline Cross_Val_Score Results (Optimal Alpha)')
cv_results

Pipeline Cross_Val_Score Results (Optimal Alpha)


{'Ridge': {'mean_R2': 0.906,
  'fold_R2': [0.8995, 0.9304, 0.8489, 0.9234, 0.9278]},
 'Lasso': {'mean_R2': 0.907,
  'fold_R2': [0.8997, 0.933, 0.8429, 0.9284, 0.9307]},
 'ElasticNet': {'mean_R2': 0.9071,
  'fold_R2': [0.8999, 0.933, 0.8437, 0.9282, 0.9307]}}